In [1]:
import os
import numpy as np
import pandas as pd
import torch
import os
import json
import pandas as pd
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2
import numpy as np
%matplotlib inline
from tqdm import tqdm
import glob
import pickle

In [15]:
!ls -lh

total 25G
-rw-rw-r-- 1 core core 5.3K Aug  8 17:40 challenge-2019-classes-description-segmentable.csv
-rw-rw-r-- 1 core core  21K Jul 26 08:26 challenge-2019-label300-segmentable-hierarchy.json
-rw-rw-r-- 1 core core 369M Aug  8 17:46 challenge-2019-train-segmentation-bbox.csv
-rw-rw-r-- 1 core core  82M Aug  8 17:47 challenge-2019-train-segmentation-labels.csv
-rw-rw-r-- 1 core core 613M Aug  8 18:03 challenge-2019-train-segmentation-masks.csv
-rw-rw-r-- 1 core core 6.4M Aug  8 18:03 challenge-2019-validation-segmentation-bbox.csv
-rw-rw-r-- 1 core core 2.4M Aug  8 18:03 challenge-2019-validation-segmentation-labels.csv
-rw-rw-r-- 1 core core 2.7M Aug  8 18:03 challenge-2019-validation-segmentation-masks.csv
-rw-rw-r-- 1 core core 543K Sep  1 19:05 check_preds.ipynb
-rw-rw-r-- 1 core core 535K Sep  2 17:36 check_preds-parent.ipynb
-rw-r--r-- 1 core core 371K Sep  2 09:43 eda.ipynb
-rw-rw-r-- 1 core core  27M Sep  2 17:42 merge_1000_0902.csv.zip
-rw-rw-r-- 1 core core  25K Sep  2 17:44

In [17]:
csv_files = [
            #'sub_first_1000_0902_1_top50.csv',
            #'sub_parent_0902_1000_1.csv',
            'sub_0902_3_top50.csv.zip',
            'sub_parent_0903_1_top50.csv'
            ]

dfs = [pd.read_csv(x) for x in csv_files]

In [18]:
for df in dfs:
    df.PredictionString = df.PredictionString.fillna('')

In [19]:
len(dfs[1].iloc[0].PredictionString)

12047

In [20]:
# lower conf for is model, not working
# dfs[1].PredictionString = dfs[1].PredictionString.map(lambda x: get_low_conf_pred_str(x))

In [21]:
dfs[0].head()

,ImageID,ImageWidth,ImageHeight,PredictionString
0,80155d58d0ee19bd,1024,683,/m/083wq 0.0024876 eNqVU01vwyAM/UsGEjb1Ou3QJGC...
1,52a76f3fe21185bf,1024,683,/m/083wq 0.0176641 eNoLyIqyzDWISzFABob+2NmUAXS...
2,65e9da7c04c8e4cd,1024,682,
3,6d10fa21d93bd6cc,1024,682,/m/05r655 0.0065765 eNrNUttSwyAQ/SUICTTNxNFOLL...
4,0841dee573a875ca,1024,683,/m/05r655 0.0021448 eNqFlMmiqjoQRX/JlL4ZjcL0JK...


In [22]:
#keep top 100
#dfs[0].PredictionString = dfs[0].PredictionString.map(lambda x: ' '.join(x.split(' ')[:1200]))

In [23]:
dfs[1].head(10)

,ImageID,ImageWidth,ImageHeight,PredictionString
0,80155d58d0ee19bd,1024,683,/m/0k4j 0.2084172 eNqNU1tuwyAQvBLLw6gnSI15KAeo...
1,52a76f3fe21185bf,1024,683,/m/0k4j 0.0547043 eNpLCQmxzDeISTFAAEN/GIlgGWDI...
2,65e9da7c04c8e4cd,1024,682,/m/02crq1 0.0013185 eNoLiDDMN4hOMYAAQ38Df0MQCW...
3,6d10fa21d93bd6cc,1024,682,/m/0k4j 0.0118669 eNpNi8EKwkAMRH9pdtFFoQfFi1TJ...
4,0841dee573a875ca,1024,683,/m/0k4j 0.0986574 eNoLC4wxSjDKTjJABYb+MBIZg0gE...
5,660329181edd20b4,1024,483,/m/01g317 0.0025120 eNp1UtFuwjAM/CU7bVaVScCmqZ...
6,2dffa0787ba35f70,1024,545,/m/01g317 0.0115226 eNpLTgwzMs9IMDD2M/SHQwMDKG...
7,cb66703ecbd0b99a,1024,678,/m/0k4j 0.0017437 eNqLDIlOMjAOTDUw8TH2M/QDYgMD...
8,2f01f2c68316b5e5,1024,768,/m/0k4j 0.1820910 eNqNU81uwyAMfiUbR9qhj9Bgq1O1...
9,36cfe3fa46f56ed3,768,768,/m/0k4j 0.0310719 eNrLzAk1MklLN7B2N/My9zb1MfID...


In [24]:
x = None
for d in dfs:
    if x is None:
        x = d.PredictionString
    else:
        x = x.str.cat(d.PredictionString, sep=' ')

In [25]:
dfs[0].PredictionString = x
print(len(dfs[0].iloc[0].PredictionString), len(dfs[1].iloc[0].PredictionString))

34068 12047


In [26]:
dfs[0].head()

,ImageID,ImageWidth,ImageHeight,PredictionString
0,80155d58d0ee19bd,1024,683,/m/083wq 0.0024876 eNqVU01vwyAM/UsGEjb1Ou3QJGC...
1,52a76f3fe21185bf,1024,683,/m/083wq 0.0176641 eNoLyIqyzDWISzFABob+2NmUAXS...
2,65e9da7c04c8e4cd,1024,682,/m/02crq1 0.0013185 eNoLiDDMN4hOMYAAQ38Df0MQC...
3,6d10fa21d93bd6cc,1024,682,/m/05r655 0.0065765 eNrNUttSwyAQ/SUICTTNxNFOLL...
4,0841dee573a875ca,1024,683,/m/05r655 0.0021448 eNqFlMmiqjoQRX/JlL4ZjcL0JK...


In [27]:
dfs[0].to_csv('merge_0903_1.csv', index=False)

In [28]:
!kaggle competitions submit -c open-images-2019-instance-segmentation -f merge_0903_1.csv -m "merge 0903"

100%|██████████████████████████████████████| 3.15G/3.15G [01:21<00:00, 41.5MB/s]
Successfully submitted to Open Images 2019 - Instance Segmentation

In [ ]:
!ls

In [11]:
len(dfs[0].loc[dfs[0].PredictionString.isna()])

10336

In [56]:
len(dfs[0].loc[dfs[0].PredictionString.isna()])

0

In [12]:
len(dfs[1].loc[dfs[1].PredictionString.isna()])

41

In [100]:
#####
lst = ['Geeks', 'For', 'Geeks', 'is',  
            'portal', 'for', 'Geeks'] 
tmp1 = pd.DataFrame(lst, columns=['t'])
lst2 = ['a', 'b', 'c', 'd',  
            'e', 'f', 'g'] 
tmp2 = pd.DataFrame(lst2, columns=['t'])

lst3 = ['a1', 'b1', 'c', 'd',  
            'e', 'f', 'g'] 
tmp3 = pd.DataFrame(lst3, columns=['t'])
tmp3.head()

,t
0,a1
1,b1
2,c
3,d
4,e


In [4]:
df_is = pd.read_csv('ensemble_is_42_0808_1_3_models.csv')
df_is.head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,0.1689330 /m/04bcr3 0.3664692 0.7250217 0.4814...
1,763c72626bf24534,0.7671249 /m/04bcr3 0.6873273 0.8193369 0.9977...
2,7540c162180973b1,0.0219995 /m/04dr76w 0.0762036 0.3234025 0.120...
3,c4e522dce52cb02f,0.4940703 /m/04bcr3 0.8307936 0.6809913 0.9920...
4,cafcd9496c808085,0.0129044 /m/04dr76w 0.0000000 0.0600080 0.031...


In [5]:
def get_low_conf_pred_str(pred_str):
    try:
        if len(pred_str) < 1:
            return pred_str
    except:
        print('pred_str:', pred_str)
        raise
    det_items = pred_str.split(' ')
    
    res = []
    for i, c in enumerate(det_items):
        if i % 12 == 0:
            res.append(str(float(c)/4))
        else:
            res.append(c)
    return ' '.join(res)